In [ ]:
def preprocess_date(data):
    data['month'], data['date'], data['year'] = data['date'].str.split('.').str
    data[['month']] = data[['month']].astype(int)
    data[['date']] = data[['date']].astype(int)
    data[['year']] = data[['year']].astype(int)
    return data


def preprocess(data):
    data = preprocess_date(data)
    
    
    return data



In [ ]:
def groupby_dbn_shopid_item(data)
    
    x_train = []
    y_train = []
    
    return x_train, y_train